1.IMPORTING NECESSARY LIBRARIES

In [ ]:
import os
import zipfile
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from sklearn.metrics import accuracy_score
import random
import shutil
from sklearn.model_selection import train_test_split

2.EXTRACTING THE DATASET

In [ ]:
import zipfile
import os

zip_path = '/content/dataset.zip'  # Update if your zip file is located elsewhere
extract_to = 'dataset'

# Extract the dataset if everything is okay
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_to)

print("Dataset extracted to:", extract_to)

Dataset extracted to: dataset


In [ ]:
data_dir = os.path.join(extract_to, '/content/dataset/Dataset_BUSI_with_GT')

3.DATASET SPLITTING

In [ ]:
# Create directories for training and testing splits
train_dir = os.path.join(extract_to, 'train')
test_dir = os.path.join(extract_to, 'test')
os.makedirs(train_dir, exist_ok=True)
os.makedirs(test_dir, exist_ok=True)

# Get list of classes (subdirectories inside `data_dir`)
classes = os.listdir(data_dir)
for cls in classes:
    os.makedirs(os.path.join(train_dir, cls), exist_ok=True)
    os.makedirs(os.path.join(test_dir, cls), exist_ok=True)

In [ ]:
# Split images into train and test sets for each class
for cls in classes:
    class_dir = os.path.join(data_dir, cls)
    images = os.listdir(class_dir)
    train_images, test_images = train_test_split(images, test_size=0.2, random_state=42)

    for img in train_images:
        shutil.move(os.path.join(class_dir, img), os.path.join(train_dir, cls, img))

    for img in test_images:
        shutil.move(os.path.join(class_dir, img), os.path.join(test_dir, cls, img))

print("Dataset split into train and test sets.")

Dataset split into train and test sets.


4.TRAINING DATA AUGMENTATION

In [ ]:
train_datagen = ImageDataGenerator(
    rescale=1.0/255,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)



In [ ]:
test_datagen = ImageDataGenerator(rescale=1.0/255)

train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary'
)



Found 1048 images belonging to 2 classes.


In [ ]:
test_generator = test_datagen.flow_from_directory(
    test_dir,
    target_size=(224, 224),
    batch_size=32,
    class_mode='binary',
    shuffle=False
)

Found 264 images belonging to 2 classes.


In [ ]:
# Model setup and training code (same as previously provided)
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model

5.LOAD AND CUSTOMIZE RESNET-50

In [ ]:
# Load the ResNet50 model pre-trained on ImageNet and fine-tune for our task
base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224, 224, 3))

# Freeze the layers in the base model
for layer in base_model.layers:
    layer.trainable = False

94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


6.ADDING CUSTOM LAYERS

In [ ]:
# Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(1024, activation='relu')(x)
predictions = Dense(1, activation='sigmoid')(x)

7.COMPILING THE MODEL

In [ ]:
# Define the final model
model = Model(inputs=base_model.input, outputs=predictions)

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

8.TRAINING THE MODEL

In [ ]:
# Train the model
epochs = 10
history = model.fit(
    train_generator,
    epochs=epochs,
    validation_data=test_generator
)

Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:122: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


33/33 ━━━━━━━━━━━━━━━━━━━━ 253s 7s/step - accuracy: 0.6195 - loss: 0.7798 - val_accuracy: 0.7689 - val_loss: 0.5248
Epoch 2/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 251s 7s/step - accuracy: 0.7796 - loss: 0.5358 - val_accuracy: 0.7955 - val_loss: 0.4807
Epoch 3/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 243s 7s/step - accuracy: 0.7772 - loss: 0.5105 - val_accuracy: 0.7689 - val_loss: 0.4818
Epoch 4/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 272s 8s/step - accuracy: 0.7204 - loss: 0.5712 - val_accuracy: 0.8030 - val_loss: 0.4587
Epoch 5/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 321s 8s/step - accuracy: 0.7677 - loss: 0.4975 - val_accuracy: 0.7273 - val_loss: 0.5116
Epoch 6/10
33/33 ━━━━━━━━━━━━━━━━━━━━ 287s 7s/step - accuracy: 0.7767 - loss: 0.5079 - val_accuracy: 0.7955 - val_loss: 0.4373
Epoch 7/10
24/33 ━━━━━━━━━━━━━━━━━━━━ 51s 6s/step - accuracy: 0.7380 - loss: 0.5303

9.EVALUATING THE MODEL

In [ ]:
# Evaluate the model on test data
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Testing Accuracy: {test_accuracy * 100:.2f}%')

9/9 ━━━━━━━━━━━━━━━━━━━━ 54s 6s/step - accuracy: 0.9144 - loss: 0.2359
Testing Accuracy: 78.03%


In [ ]:
from tensorflow.keras import layers,models

10.SAVING THE MODEL

In [ ]:
model.save('resnet_model.h5')